In [1]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize

import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

# For Python2 this have to be done
# from __future__ import division


### Import Toronto data as df,  userJson as df_user

In [13]:
df = pd.read_csv('../data/toronto_reviews.csv')

In [3]:
reviewJson = '../data/Export_CleanedReview.json'
# file directory
reviewJson, file_user = [
    '..\\data\\Export_CleanedReview.json',
    '..\\data\\user.json',
]

In [ ]:
# with open(reviewJson,'r') as f:
#     data = f.readlines()
#     data = list(map(json.loads, data))
# df_review = pd.DataFrame(data[0])

In [4]:
with open(file_user, encoding = 'utf-8') as json_file:
    data = json_file.readlines()
    data = list(map(json.loads, data))

df_user = pd.DataFrame(data)

### Data Processing

In [14]:
# Rename Month as Day and Day as Month(error occurs on original data)
df=df.rename(columns = {'Month':'new_Day'})
df=df.rename(columns = {'Day':'Month'})
df=df.rename(columns = {'new_Day':'Day'})

In [15]:
# Merge the Year, Month Day as to one column
df['Date'] = pd.to_datetime(df[['Year', 'Month','Day']].assign(DAY=1))

In [ ]:
# df.columns.values

In [16]:
# Drop the unuseful columns
df.drop(columns=['Year', 'Month','Day','Unnamed: 0','Unnamed: 0.1','friend_count','ghost','img_dsc','img_url','nr','photo_count','review_count','review_date','review_language','ufc','user_loc','vote_count','Updated'],inplace=True)

In [17]:
# Inner join the df with df_user to get the average_stars column
df = pd.merge(df, df_user[['average_stars','user_id']], on='user_id', how='inner')

In [18]:
df

,business_id,rating,review_id,review_text,user_id,Date,average_stars
0,bSQGCheX1BwvL67Az1OJlA,5.0,NjYpdf2W7_2UU691w9Rlnw,I had a good experience at this store. I was g...,q41qqkChL9fRxF43cnkgbw,2018-09-01,4.29
1,DFFiZZyCjciD_j1wgM3N0w,5.0,KlzAeONIVlEe43TqsGUZXw,"Great, quick, and easy experience with Sheldon...",q41qqkChL9fRxF43cnkgbw,2017-12-01,4.29
2,j7_yOwPVGPrzsF6uNOngEg,4.0,XfoUEjgn7fXDXkmyfaqJwA,"Pretty small store and selection is limited, b...",q41qqkChL9fRxF43cnkgbw,2017-11-01,4.29
3,_4B99coJlSasAgQPm0gyqA,5.0,rXKs_11BT3s7KiQ2rlcPaA,It was our first time using Wild North Flowers...,q41qqkChL9fRxF43cnkgbw,2017-12-01,4.29
4,W2W3tTQkid1Z5PXTnVQP6Q,5.0,YMotxam_oSk6uFEPkVYvLg,I have been to Reptilia with both adults and l...,q41qqkChL9fRxF43cnkgbw,2018-08-01,4.29
5,_cjPEH9wXhKS-HQe_U3M4Q,3.0,s2AuZqdXrND_LtMh7_YeZA,This was maybe my fourth or fifth time at this...,q41qqkChL9fRxF43cnkgbw,2018-08-01,4.29
6,RA00GtlU25TkIP9i9wd8bg,3.0,ilbELhGN4eQw9PYOO7gFFg,Super pricey. While some of the exhibits were ...,q41qqkChL9fRxF43cnkgbw,2017-11-01,4.29
7,gnCl7VXV5BALM0SqOHHmGg,5.0,0YRlmlpv_K-_Bt8VBl1E2w,We followed Dr. Kulik from her previous office...,q41qqkChL9fRxF43cnkgbw,2018-08-01,4.29
8,Tga64GlSy6xjp-Z-Ogddng,4.0,_06CaWLnmbo9P9C2NckuQQ,We have been customers of crate and barrel for...,q41qqkChL9fRxF43cnkgbw,2018-08-01,4.29
9,WkdIxfgIj-iH1OL4YJ6oFQ,5.0,EGR9C15IhlPkrPjXYZG9yA,Super simple process. I was looking for a rela...,q41qqkChL9fRxF43cnkgbw,2018-08-01,4.29


In [19]:
Export = df.to_json(r'..\\data\\Export_TorontoData.json', orient='records')